In [1]:
import requests
import yfinance as yf
import yahooquery as yq
import pandas as pd
import sqlite3
import numpy as np
import plotly.graph_objs as go
from scipy.optimize import minimize
from plotly.subplots import make_subplots
import scipy

In [2]:
adj_close=[]
pd.concat(adj_close,axis=1)

ValueError: No objects to concatenate

In [6]:
import os

# Print current working directory
print(os.getcwd())

# Change current working directory to /path/to/new/directory
os.chdir("./3.25/bigbucks")

# Print current working directory
print(os.getcwd())

/Users/qijunyang/Documents/GitLab/fintech512-bigbucks/fintech512-bigbucks
/Users/qijunyang/Documents/GitLab/fintech512-bigbucks/fintech512-bigbucks/3.25/bigbucks


In [4]:
ssymbol='MSFT'
conn = sqlite3.connect('./bigbucks.sqlite')
cur=conn.execute("SELECT symbol, date, close FROM stocks_price_data WHERE symbol = '{}'".format(ssymbol))
data=cur.fetchall()
df=pd.DataFrame(data,columns=['symbol','date','close'])
# df.sort_values('date',ascending=False,inplace=True)
# df['rts']=df['close'].pct_change().fillna(0)
# data = []
# df.apply(lambda row: data.append({'stock':row[0], 'date': row[1], 'returns': row[3]}),axis=1)
# data

# import plotly.express as px
# fig = px.bar(df, x='date', y='rts')
# fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
#                   marker_line_width=1.5, opacity=0.6)
# fig.show()


In [3]:
ssymbol='SPY'
conn = sqlite3.connect('./bigbucks.sqlite')
cur=conn.execute("SELECT symbol, date, close FROM stocks_price_data WHERE symbol = '{}'".format(ssymbol))
data=cur.fetchall()
df=pd.DataFrame(data,columns=['symbol','date','close'])
import plotly.express as px
fig = px.bar(df, x='date', y='close')
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.show()
df['close'].name=df['symbol'][0]
df

symbol        date       close
0       SPY  2018-04-09  239.677719
1       SPY  2018-04-10  243.488708
2       SPY  2018-04-11  242.212280
3       SPY  2018-04-12  244.205032
4       SPY  2018-04-13  243.488708
...     ...         ...         ...
1254    SPY  2023-03-31  409.390015
1255    SPY  2023-04-03  410.950012
1256    SPY  2023-04-04  408.670013
1257    SPY  2023-04-05  407.600006
1258    SPY  2023-04-06  409.190002

[1259 rows x 3 columns]

In [ ]:
db=get_db()
price=db.execute("SELECT symbol, date, close FROM stocks_price_data WHERE symbol = '{}'".format(ssymbol))
data = []
for row in price:
    data.append({'stock':row[0], 'date': row[1], 'close': row[2]})

df=pd.DataFrame(price,columns=['symbol','date','close'])
df.sort_values('date',ascending=False,inplace=True)
df['rts']=df['close'].pct_change().fillna(0)
print(df)

In [7]:
symbols = ['SPY', 'MSFT']
query = "SELECT symbol, date, close FROM stocks_price_data WHERE symbol IN ({seq})".format(seq=','.join(['?']*len(symbols)))
data=conn.execute(query,symbols).fetchall()
df=pd.DataFrame(data,columns=['symbol','date','close'])
df.set_index('date',inplace=True)
data=[]
for symbol in symbols:
    temp=df.query("symbol==@symbol")['close']
    temp.name=symbol
    data.append(temp)
df=pd.concat(data,axis=1)
df.sort_index(ascending=True,inplace=True)
rts=df.apply(lambda price: price.pct_change().fillna(0))
rts.columns=[symbol+'_rt' for symbol in symbols]
cum_rts=(rts+1).cumprod()
cum_rts.columns=[symbol+'_cum_rt' for symbol in symbols]

df=pd.concat([df,rts,cum_rts],axis=1)
df['pre_rt']=df[ symbols[1]+'_rt' ].shift().fillna(0)
# df['date']=df.index


# fig = px.line(df, x='date', y=df.columns,
#               hover_data={"date": "|%B %d, %Y"},
#               title='custom tick labels with ticklabelmode="period"')
# fig.update_layout(
#     title='Historical Price',
#     xaxis_title='Date',
#     yaxis_title='Price',
#     autosize=True,
#     title_x=0.5,
# )
# fig.update_xaxes(rangeslider_visible=True)
# plot_div=fig.to_html(full_html=False)
# fig.show()

In [16]:
ssymbol='MSFT'
conn = sqlite3.connect('./bigbucks.sqlite')
data=conn.execute("SELECT * FROM stocks_price_data WHERE symbol = '{}'".format(ssymbol)).fetchall()
df=pd.DataFrame(data,columns=['symbol','date','open','high','low','close','volume'])
fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()

In [86]:
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['AAPL.Open'],
                high=df['AAPL.High'],
                low=df['AAPL.Low'],
                close=df['AAPL.Close'])])

fig.show()

date
2018-04-09    0.000000
2018-04-10    0.023246
2018-04-11   -0.010982
2018-04-12    0.018724
2018-04-13   -0.005343
                ...   
2023-03-31    0.014962
2023-04-03   -0.003711
2023-04-04   -0.000174
2023-04-05   -0.009889
2023-04-06    0.025533
Name: MSFT_rt, Length: 1259, dtype: float64


In [82]:
import plotly.figure_factory as ff
import numpy as np
hist_data = [df[symbols[1]+'_rt'].values]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels)
fig.show()

In [84]:
fig = px.histogram(df, x=symbols[1]+'_rt')
fig.show()

In [55]:
df = px.data.stocks(indexed=True)-1
df

company         GOOG      AAPL      AMZN        FB      NFLX      MSFT
date                                                                  
2018-01-01  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
2018-01-08  0.018172  0.011943  0.061881 -0.040032  0.053526  0.015988
2018-01-15  0.032008  0.019771  0.053240 -0.029757  0.049860  0.020524
2018-01-22  0.066783 -0.019943  0.140676  0.016858  0.307681  0.066561
2018-01-29  0.008773 -0.082857  0.163374  0.018357  0.273537  0.040708
...              ...       ...       ...       ...       ...       ...
2019-12-02  0.216280  0.546914  0.425061  0.075997  0.463641  0.720717
2019-12-09  0.222821  0.572286  0.432660  0.038855  0.421496  0.752239
2019-12-16  0.224418  0.596800  0.453455  0.104094  0.604362  0.784896
2019-12-23  0.226504  0.656000  0.521226  0.113728  0.567170  0.802472
2019-12-30  0.213014  0.678000  0.503360  0.098475  0.540883  0.788185

[105 rows x 6 columns]

In [ ]:
def plot_index_price(ssymbol):
    db=get_db()
    symbols = ['SPY', ssymbol]
    query = "SELECT symbol, date, close FROM stocks_price_data WHERE symbol IN ({seq})".format(seq=','.join(['?']*len(symbols)))
    data=db.execute(query,symbols).fetchall()
    df=pd.DataFrame(data,columns=['symbol','date','close'])
    df.set_index('date',inplace=True)
    data=[]
    for symbol in symbols:
        temp=df.query("symbol==@symbol")['close']
        temp.name=symbol
        data.append(temp)
    df=pd.concat(data,axis=1)
    df['date']=df.index

    fig = px.line(df, x='date', y=df.columns,
                hover_data={"date": "|%B %d, %Y"},
                title='custom tick labels with ticklabelmode="period"')
    fig.update_layout(
        title='Historical Price',
        xaxis_title='Date',
        yaxis_title='Price',
        autosize=True,
        title_x=0.5,
    )
    fig.update_xaxes(rangeslider_visible=True)
    plot_div=fig.to_html(full_html=False)
    return render_template('stock_plot.html', plot_div=plot_div)

In [ ]:
ssymbol='MSFT'
db = sqlite3.connect('./bigbucks.sqlite')
data=db.execute("SELECT * FROM stocks_price_data WHERE symbol = '{}'".format(ssymbol)).fetchall()

In [46]:
db = sqlite3.connect('./bigbucks.sqlite')
all_stocks= db.execute('SELECT DISTINCT stock_id FROM portfolios where user_id= ?',(1,)).fetchall()
symbols = []
for stock in all_stocks:
    symbol = db.execute('SELECT symbol FROM stocks where id = ?', (stock[0],)).fetchone()
    symbols.append(symbol[0])
adj_close=[]
for symbol in symbols:
    data = db.execute('SELECT date,close FROM stocks_price_data WHERE symbol = ?',(symbol,)).fetchall()
    close = pd.DataFrame(data,columns=['date',symbol]).set_index(['date'])
    adj_close.append(close)



adj_close = pd.concat(adj_close,axis=1).sort_index(ascending=True)
# Compute pairwise covariance of columns, excluding NA/null values.
cov = adj_close.pct_change(fill_method=None).cov()*252 # Annual
try:
    np.linalg.cholesky(cov)
except:
    cov=pd.DataFrame(near_psd(cov.to_numpy(),np.eye(n)).psd,columns=cov.columns,index=cov.index)
# Percentage change between the current and a prior element and fill NAN with NAN
expected_rts = adj_close.pct_change(fill_method=None).mean()*252 # Annual
n = expected_rts.shape[0]

def calculate_portfolio_vol(weights):
    return np.sqrt(weights@cov@weights)


bounds = [(0, 1) for i in range(n)]
initial_weights = np.ones(n) / n
efficient_frontier = []
rf=0.025
if n==1:
    efficient_frontier = []
    for idx,(rt,std) in enumerate(zip(np.linspace(1e-10,expected_rts[0],20),np.linspace(1e-10,cov.iloc[0,0],20))):
        efficient_frontier.append((rt,std,(rt-rf)/std,1/(20-1)*idx))
    idx=pd.DataFrame(efficient_frontier).iloc[:,2].idxmax()
    maxSR_rt=str(round(efficient_frontier[idx][0]*100,2))+'%'
    maxSR_std=str(round(efficient_frontier[idx][1]*100,2))+'%'
    maxSR=round(efficient_frontier[idx][2],2)
    maxSR_w=pd.DataFrame([efficient_frontier[idx][3]],index=expected_rts.index).T.apply(lambda x: "{:.2f}%".format(x[0]*100))
    efficient_frontier.append((maxSR_rt,maxSR_std,maxSR,maxSR_w))
else:
    for target_return in np.linspace(0.02,0.5, 50):
        constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1},{'type': 'eq', 'fun': lambda w: w@expected_rts - target_return}]
        result = minimize(
            calculate_portfolio_vol,
            initial_weights,
            method="SLSQP",
            bounds=bounds,
            constraints=constraints,
            options={"maxiter": 1000},
            args=(),
        )
        # if not result.success:
        #     # print(result)
        #     print('-----------------------------------')
        #     continue
        weights = result.x
        portfolio_return = weights@expected_rts
        portfolio_std_dev = np.sqrt(calculate_portfolio_vol(weights))
        efficient_frontier.append((portfolio_return, portfolio_std_dev,(portfolio_return-rf)/portfolio_std_dev,weights))


    idx=pd.DataFrame(efficient_frontier)[2].idxmax()
    maxSR_rt=round(efficient_frontier[idx][0],2)
    maxSR_std=round(efficient_frontier[idx][1],2)
    maxSR=round(efficient_frontier[idx][2],2)
    maxSR_w=efficient_frontier[idx][3]

In [12]:
db = sqlite3.connect('./bigbucks.sqlite')
all_stocks= db.execute('SELECT DISTINCT stock_id,shares_owned FROM portfolios where user_id= ?',(1,)).fetchall()
symbols = []
shares_owned=[]
for stock in all_stocks:
    symbol = db.execute('SELECT symbol FROM stocks where id = ?', (stock[0],)).fetchone()
    symbols.append(symbol[0])
    shares_owned.append(stock[1])

shares_owned=np.array(shares_owned)/sum(shares_owned)
currentRt=shares_owned@expected_rts
currentVol=calculate_portfolio_vol(shares_owned)
currentPositionInEF=(currentRt,currentVol,(currentRt-rf)/currentVol,shares_owned)

NameError: name 'expected_rts' is not defined

In [15]:
db = sqlite3.connect('./bigbucks.sqlite')
all_stocks= db.execute('SELECT DISTINCT stock_id,shares_owned FROM portfolios where user_id= ?',(1,)).fetchall()
symbols = []
shares_owned=[]
for stock in all_stocks:
    symbol = db.execute('SELECT symbol FROM stocks where id = ?', (stock[0],)).fetchone()
    symbols.append(symbol[0])
    shares_owned.append(stock[1])
shares_owned=np.array(shares_owned)/sum(shares_owned)


adj_close=[]
for symbol in symbols:
    data = db.execute('SELECT date,close FROM stocks_price_data WHERE symbol = ?',(symbol,)).fetchall()
    close = pd.DataFrame(data,columns=['date',symbol]).set_index(['date'])
    adj_close.append(close)



adj_close = pd.concat(adj_close,axis=1).sort_index(ascending=True)
# Compute pairwise covariance of columns, excluding NA/null values.
cov = adj_close.pct_change(fill_method=None).cov()*252 # Annual
try:
    np.linalg.cholesky(cov)
except:
    cov=pd.DataFrame(near_psd(cov.to_numpy(),np.eye(n)).psd,columns=cov.columns,index=cov.index)
# Percentage change between the current and a prior element and fill NAN with NAN
expected_rts = adj_close.pct_change(fill_method=None).mean()*252 # Annual
n = expected_rts.shape[0]

def calculate_portfolio_vol(weights):
    return np.sqrt(weights@cov@weights)


bounds = [(0, 1) for i in range(n)]
initial_weights = np.ones(n) / n
efficient_frontier = []
rf=0.025
max_rt=expected_rts.max()

currentRt=shares_owned@expected_rts
currentVol=calculate_portfolio_vol(shares_owned)
shares_owned=pd.DataFrame(shares_owned,index=expected_rts.index).T.apply(lambda x: "{:.2f}%".format(x[0]*100))
currentPositionInEF=(currentRt,currentVol,(currentRt-rf)/currentVol,shares_owned)

for target_return in np.linspace(0,max_rt, 50):
    constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1},{'type': 'eq', 'fun': lambda w: w@expected_rts - target_return}]
    result = minimize(
        calculate_portfolio_vol,
        initial_weights,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": 1000},
        args=(),
    )
    # if not result.success:
    #     # print(result)
    #     print('-----------------------------------')
    #     continue
    weights = result.x
    portfolio_return = weights@expected_rts
    portfolio_std_dev = calculate_portfolio_vol(weights)
    efficient_frontier.append((portfolio_return, portfolio_std_dev,(portfolio_return-rf)/portfolio_std_dev,weights))


# idx=pd.DataFrame(efficient_frontier).iloc[:,2].idxmax()
# maxSR_rt=str(round(efficient_frontier[idx][0]*100,2))+'%'
# maxSR_std=str(round(efficient_frontier[idx][1]*100,2))+'%'
# maxSR=round(efficient_frontier[idx][2],2)
# maxSR_w=pd.DataFrame(efficient_frontier[idx][3],index=expected_rts.index).T.apply(lambda x: "{:.2f}%".format(x[0]*100))

# efficient_frontier.append((maxSR_rt,maxSR_std,maxSR,maxSR_w))

In [16]:
currentPositionInEF

(0.32784436212228785,
 0.3337927508246779,
 0.9072826218486527,
 AAPL    100.00%
 dtype: object)

In [6]:

def get_quantity_owned(stock_id):
    db = sqlite3.connect('./bigbucks.sqlite')
    quantity_owned = db.execute('SELECT shares_owned from portfolios WHERE stock_id like ?', (stock_id,)).fetchone()
    if quantity_owned == None:
        return 0
    else:
        return quantity_owned[0]

In [7]:
db = sqlite3.connect('./bigbucks.sqlite')
all_stocks= db.execute('SELECT DISTINCT stock_id FROM portfolios where user_id= ?',(1,)).fetchall()
stocks = []
portfolioBalance=0
if all_stocks != None:
    for stock in all_stocks:
        stock_data = db.execute('SELECT shortName, symbol FROM stocks where id = ?', (stock[0],)).fetchone()
        if stock_data != None:
            stocks.append({
                'stock_id': stock[0],
                'name': stock_data[0],
                'symbol': stock_data[1],
                'shares_owned': get_quantity_owned(stock[0]),
            })
    for stock in stocks:
        fast_info = yf.Ticker(stock['symbol']).fast_info
        price=round(fast_info['lastPrice'],2) if fast_info['lastPrice']!=None else 0
        portfolioBalance+=stock['shares_owned']*price

In [8]:
portfolioBalance

1983.8899999999996

In [192]:
test=pd.DataFrame(maxSR_w,index=expected_rts.index).T.apply(lambda x: "{:.2f}%".format(x[0]*100))
test['MSFT']

'0.00%'

In [130]:
class Weighted_F_norm:
    '''
    Given the weight matrix, calculate the Weighted Frobenius Norm. (Assume it's diagonal)
    '''
    def compare_F(self,mat_a,mat_b,mat_w):
        '''Give two matrix, use Weighted Frobenius Norm to calculate how close they are'''
        err = mat_a-mat_b #difference
        weighted_err = np.sqrt(mat_w) @ err @ np.sqrt(mat_w) 
        w_F_norm = np.sqrt(np.square(weighted_err).sum())
        return w_F_norm
    
    def calculate_F(self,mat,mat_w):
        "Given one matrix, calculate its Weighted Frobenius Norm"
        weighted_err = np.sqrt(mat_w) @ mat @ np.sqrt(mat_w)
        w_F_norm = np.sqrt(np.square(weighted_err).sum())
        return w_F_norm

class near_psd(Weighted_F_norm):
    '''
    Rebonato and Jackel's Method to get acceptable PSD matrix 
    
    Parameters:
        not_psd -- the matrix which is not positive semi-definite matrix
        weight  -- used for calculating the Weighted Frobenius Norm (Assume it's diagonal)

    Usage:
        near_psd_model=near_psd(non_psd,weight)
        psd=near_psd_model.psd
    '''
    # initialization
    def __init__(self,not_psd,weight):
        self.__not_psd=not_psd
        self.__weight=weight
        self.run() # main function
        self.F_compare_norm(weight) # Weighted Frobenius Norm
        
    def run(self):
        n=self.__not_psd.shape[0]
        # Set the weight matrix to be identity matrix
        invSD = np.eye(n)
        corr=self.__not_psd
        # if the matrix is not correlation matrix, convert it to the correlation matrix
        if not np.allclose(np.diag(self.__not_psd),np.ones(n)):
            invSD=np.diag(1/np.sqrt(np.diag(self.__not_psd)))
            corr=invSD @ self.__not_psd @ invSD
        eig_val,eig_vec=np.linalg.eigh(corr) # eigenvalues & eigenvectors 
        eig_val[eig_val<0]=0 # adjust the negative value to 0
        # get the scale matrix
        scale_mat = np.diag(1/(eig_vec * eig_vec @ eig_val))
        B = np.sqrt(scale_mat) @ eig_vec @ np.sqrt(np.diag(eig_val))
        corr=B @ B.T
        # convert it back into original form
        SD=np.diag(1/np.diag(invSD))
        psd = SD @ corr @ SD
        self.__psd = psd

    # Weighted Frobenius Norm of the difference between near_psd and ono_psd
    def F_compare_norm(self,weight):
        self.__F = self.compare_F(self.__psd,self.__not_psd,weight)

    @property
    def psd(self):
        return self.__psd
    
    @property
    def F(self):
        return self.__F 

In [197]:
def efficient_frontier():
    # db=get_db()
    db = sqlite3.connect('./bigbucks.sqlite')

    # g.user['id']=1
    all_stocks= db.execute('SELECT DISTINCT stock_id FROM portfolios where user_id= ?',(1,)).fetchall()
    symbols = []
    for stock in all_stocks:
        symbol = db.execute('SELECT symbol FROM stocks where id = ?', (stock[0],)).fetchone()
        symbols.append(symbol[0])
    if symbols==[]:
        return
    efficient_frontier = calculate_efficient_frontier(symbols)
    fig = create_efficient_frontier_chart(efficient_frontier)
    ans={}
    ans['plot_div']=fig.to_html(full_html=False)
    ans['optimal_portfolio']=efficient_frontier[-1]
    # return render_template('efficient_frontier.html', plot_div=plot_div)
    fig.show()
    return ans

def calculate_efficient_frontier(symbols):
    # db = get_db()
    db = sqlite3.connect('./bigbucks.sqlite')
    # risk-free rate
    rf=0.025
    adj_close=[]
    for symbol in symbols:
        data = db.execute('SELECT date,close FROM stocks_price_data WHERE symbol = ?',(symbol,)).fetchall()
        close = pd.DataFrame(data,columns=['date',symbol]).set_index(['date'])
        adj_close.append(close)
    adj_close = pd.concat(adj_close,axis=1).sort_index(ascending=True)
    # Compute pairwise covariance of columns, excluding NA/null values.
    cov = adj_close.pct_change(fill_method=None).cov()*252 # Annual
    try:
        np.linalg.cholesky(cov)
    except:
        cov=pd.DataFrame(near_psd(cov.to_numpy(),np.eye(n)).psd,columns=cov.columns,index=cov.index)
    # Percentage change between the current and a prior element and fill NAN with NAN
    expected_rts = adj_close.pct_change(fill_method=None).mean()*252 # Annual
    n = expected_rts.shape[0]

    if n==1:
        efficient_frontier = []
        for idx,(rt,std) in enumerate(zip(np.linspace(1e-10,expected_rts[0],20),np.linspace(1e-10,cov,20))):
            efficient_frontier.append((rt,std,(rt-rf)/std,2/(20-1)*idx))
        idx=pd.DataFrame(efficient_frontier).iloc[:,2].idxmax()
        maxSR_rt=round(efficient_frontier[idx][0],2)
        maxSR_std=round(efficient_frontier[idx][1],2)
        maxSR=round(efficient_frontier[idx][2],2)
        maxSR_w=efficient_frontier[idx][3]
        efficient_frontier.append((maxSR_rt,maxSR_std,maxSR,maxSR_w))
        return efficient_frontier
    
    def calculate_portfolio_vol(weights):
        return np.sqrt(weights@cov@weights)
    
    
    bounds = [(0, 1) for i in range(n)]
    initial_weights = np.ones(n) / n
    efficient_frontier = []

    for target_return in np.linspace(0.02,0.5, 50):
        constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1},{'type': 'eq', 'fun': lambda w: w@expected_rts - target_return}]
        result = minimize(
            calculate_portfolio_vol,
            initial_weights,
            method="SLSQP",
            bounds=bounds,
            constraints=constraints,
            options={"maxiter": 1000},
            args=(),
        )
        if not result.success:
            # print(result)
            print('-----------------------------------')
            continue
        weights = result.x
        portfolio_return = weights@expected_rts
        portfolio_std_dev = np.sqrt(calculate_portfolio_vol(weights))
        efficient_frontier.append((portfolio_return, portfolio_std_dev,(portfolio_return-rf)/portfolio_std_dev,weights))

    idx=pd.DataFrame(efficient_frontier)[2].idxmax()
    maxSR_rt=round(efficient_frontier[idx][0],2)
    maxSR_std=round(efficient_frontier[idx][1],2)
    maxSR=round(efficient_frontier[idx][2],2)
    maxSR_w=efficient_frontier[idx][3]

    efficient_frontier.append((maxSR_rt,maxSR_std,maxSR,maxSR_w))
    return efficient_frontier




def create_efficient_frontier_chart(efficient_frontier):
    fig = make_subplots(rows=1, cols=1, specs=[[{"type": "scatter"}]])
    fig.add_trace(
        go.Scatter(
            x=[p[1] for p in efficient_frontier],
            y=[p[0] for p in efficient_frontier],
            mode='lines',
            name='Efficient Frontier'
        ),
        row=1,
        col=1
    )

    fig.add_trace(
        go.Scatter(
            x=[p for p in np.linspace(0, efficient_frontier[-1][1])],
            y=[p for p in np.linspace(rf, efficient_frontier[-1][0])],
            mode='lines',
            name='Captial Market Line'
        ),
        row=1,
        col=1
    )

    # fig.add_trace(
    #     go.Scatter(
    #         x=[p for p in np.arange(0,1,0.1)],
    #         y=[p for p in np.arange(0,1,0.1)],
    #         mode='lines',
    #         name='Test'
    #     ),
    #     row=1,
    #     col=1
    # )

    fig.update_layout(
        title='Efficient Frontier',
        xaxis_title='Standard Deviation',
        yaxis_title='Return',
        autosize=True,
        title_x=0.5,
    )
    return fig

In [198]:
efficient_frontier()

{'plot_div': '<div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: \'local\'};</script>\n        <script type="text/javascript">/**\n* plotly.js v2.12.1\n* Copyright 2012-2022, Plotly, Inc.\n* All rights reserved.\n* Licensed under the MIT license\n*/\n!function(t){if("object"==typeof exports&&"undefined"!=typeof module)module.exports=t();else if("function"==typeof define&&define.amd)define([],t);else{("undefined"!=typeof window?window:"undefined"!=typeof global?global:"undefined"!=typeof self?self:this).Plotly=t()}}((function(){return function t(e,r,n){function i(o,s){if(!r[o]){if(!e[o]){var l="function"==typeof require&&require;if(!s&&l)return l(o,!0);if(a)return a(o,!0);var c=new Error("Cannot find module \'"+o+"\'");throw c.code="MODULE_NOT_FOUND",c}var u=r[o]={exports:{}};e[o][0].call(u.exports,(function(t){return i(e[o][1][t]||t)}),u,u.exports,t,e,r,n)}return r[o].exports}for(var a="function"==typeof require&&require,o=0;o<n.length;o++

In [201]:
float('1%'.split('%')[0])

1.0

In [70]:
import sqlite3
# Connect to database
conn = sqlite3.connect('./bigbucks.sqlite')

# Create a cursor
cursor = conn.cursor()

# Get the names of all tables
id=1
cursor.execute('SELECT * FROM stocks')
rows = cursor.fetchall()

# Print the names of all tables
print("Tables in the database:")
for row in rows:
    print(row)

    # conn = sqlite3.connect('./instance/flaskr.sqlite')
    # cur=conn.execute("SELECT stock_symbol, closing_date, close_price FROM stock_data WHERE stock_symbol = '{}'".format(ssymbol))
    # results = []
    # for row in cur:
    #     results.append({'stock':row[0], 'date': row[1], 'close': row[2]})
    # conn.close()

Tables in the database:
(1, 'MSFT', 'Microsoft Corporation', 'Technology', 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related Client Access Licenses; GitHub that provides a collaboration platform and code hosting service for developers; Nuance provides healthcare and enterprise AI solutions; and Azure, a cloud platform. It also offers enterprise support, Microsoft consul

In [ ]:
all_stocks= db.execute('SELECT DISTINCT stock_id FROM portfolios where user_id= ?',(g.user['id'],)).fetchall()
symbols = []
for stock in all_stocks:
    symbol = db.execute('SELECT symbol FROM stocks where id = ?', (stock[0],)).fetchone()
    symbols.append(symbol)

In [ ]:
@bp.route('/efficient_frontier')
def efficient_frontier():

    all_stocks= db.execute('SELECT DISTINCT stock_id FROM portfolios where user_id= ?',(g.user['id'],)).fetchall()
    symbols = []
    for stock in all_stocks:
        symbol = db.execute('SELECT symbol FROM stocks where id = ?', (stock[0],)).fetchone()
        symbols.append(symbol)
        
    efficient_frontier = calculate_efficient_frontier(symbols)
    fig = create_efficient_frontier_chart(efficient_frontier)
    plot_div = fig.to_html(full_html=False)
    return render_template('efficient_frontier.html', plot_div=plot_div)

def calculate_efficient_frontier(symbols):
    db = get_db()
    adj_close=[]
    for symbol in symbols:
        cursor = db.execute('SELECT date,close FROM stocks_price_data WHERE symbol = ?',(symbol,))
        close = pd.DataFrame(cursor.fetchall(),columns=['date',symbol]).set_index(['date'])
        adj_close.append(close)
    adj_close=pd.concat(adj_close,axis=1).sort_index(ascending=True).dropna(axis=1)
    rts=adj_close.pct_change().dropna()

    cov = np.cov(rts.T)*252
    expected_rts=rts.mean()*252
    n = expected_rts.shape[0]

    def calculate_portfolio_vol(weights):
        return np.sqrt(weights@cov@weights)
    
    
    bounds = [(0, 1) for i in range(n)]
    initial_weights = np.ones(n) / n
    efficient_frontier = []

    for target_return in np.linspace(0,0.2, 50):
        constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1},{'type': 'eq', 'fun': lambda w: w@expected_rts - target_return}]
        result = minimize(
            calculate_portfolio_vol,
            initial_weights,
            method="SLSQP",
            # bounds=bounds,
            constraints=constraints,
            # options={"maxiter": 1000},
            args=(),
        )
        if not result.success:
            continue
        weights = result.x
        portfolio_return = weights@expected_rts
        portfolio_std_dev = np.sqrt(calculate_portfolio_vol(weights))
        efficient_frontier.append((portfolio_return, portfolio_std_dev))

    return efficient_frontier

def create_efficient_frontier_chart(efficient_frontier):
    fig = make_subplots(rows=1, cols=1, specs=[[{"type": "scatter"}]])
    fig.add_trace(
        go.Scatter(
            x=[p[1] for p in efficient_frontier],
            y=[p[0] for p in efficient_frontier],
            mode='lines',
            name='Efficient Frontier'
        ),
        row=1,
        col=1
    )
    fig.update_layout(
        title='Efficient Frontier',
        xaxis_title='Standard Deviation',
        yaxis_title='Return',
        autosize=True,
        title_x=0.5,
    )
    return fig
